# Job Profitability - Exploratory Data Analysis

This notebook explores the job task dataset to understand profitability patterns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from analysis import (
    load_data, compute_job_summary, compute_task_summary,
    compute_category_summary, get_available_fiscal_years
)

In [ ]:
# Load data
DATA_PATH = '../data/Quoted_Task_Report_FY26.xlsx'
df = load_data(DATA_PATH)
print(f"Loaded {len(df):,} task records")
print(f"Columns: {len(df.columns)}")
df.head()

In [ ]:
# Check available fiscal years
fy_years = get_available_fiscal_years(df)
print(f"Fiscal Years in data: {fy_years}")
print(f"\nFY distribution:")
df['FY_Label'].value_counts().sort_index()

In [ ]:
# Compute summaries
job_summary = compute_job_summary(df)
task_summary = compute_task_summary(df)
category_summary = compute_category_summary(job_summary)

print(f"Unique Jobs: {len(job_summary):,}")
print(f"Unique Tasks: {len(task_summary):,}")
print(f"Categories: {len(category_summary)}")

## Overall Metrics

In [ ]:
total_quoted = job_summary['Quoted_Amount'].sum()
total_billable = job_summary['Billable_Amount'].sum()
total_cost = job_summary['Actual_Cost'].sum()
total_profit = job_summary['Profit'].sum()

print(f"Total Quoted:   ${total_quoted:>15,.0f}")
print(f"Total Billable: ${total_billable:>15,.0f}")
print(f"Total Cost:     ${total_cost:>15,.0f}")
print(f"Total Profit:   ${total_profit:>15,.0f}")
print(f"Overall Margin: {total_profit/total_billable*100:>15.1f}%" if total_billable > 0 else "N/A")

In [ ]:
# Jobs over budget and at loss
print(f"Jobs over budget: {job_summary['Is_Overrun'].sum():,} / {len(job_summary):,}")
print(f"Jobs at loss: {job_summary['Is_Loss'].sum():,}")

## Category Analysis

In [ ]:
# View all categories sorted by margin
category_summary.sort_values('Margin_Pct', ascending=True)[[
    'Category', 'Job_Count', 'Billable_Amount', 'Profit', 'Margin_Pct', 'Overrun_Rate'
]]

In [ ]:
# Margin by category chart
fig, ax = plt.subplots(figsize=(12, 8))
cat_sorted = category_summary.sort_values('Margin_Pct')
colors = ['red' if m < 20 else 'green' for m in cat_sorted['Margin_Pct']]
cat_sorted.plot.barh(x='Category', y='Margin_Pct', ax=ax, color=colors, legend=False)
ax.set_xlabel('Margin %')
ax.set_title('Profit Margin by Job Category')
ax.axvline(x=20, color='orange', linestyle='--', label='20% threshold')
ax.axvline(x=0, color='black', linestyle='-')
plt.tight_layout()
plt.show()

## Jobs with Lowest Margins

In [ ]:
job_summary.nsmallest(20, 'Margin_Pct')[[
    'Job_No', 'Job_Name', 'Client', 'Category',
    'Quoted_Amount', 'Billable_Amount', 'Actual_Cost',
    'Profit', 'Margin_Pct'
]]

## Unquoted Tasks (Scope Creep)

In [ ]:
unquoted = task_summary[task_summary['Is_Unquoted']]
print(f"Unquoted tasks: {len(unquoted):,}")
print(f"Total cost of unquoted work: ${unquoted['Actual_Cost'].sum():,.0f}")
print(f"\nTop 15 unquoted tasks by cost:")
unquoted.nlargest(15, 'Actual_Cost')[[
    'Job_No', 'Job_Name', 'Task_Name', 'Actual_Hours', 'Actual_Cost'
]]

## Hour Variance Distribution

In [ ]:
# Filter to jobs with quotes
quoted_jobs = job_summary[job_summary['Quoted_Hours'] > 0].copy()

fig, ax = plt.subplots(figsize=(10, 5))
quoted_jobs['Hours_Variance_Pct'].clip(-100, 200).hist(bins=50, ax=ax, color='steelblue')
ax.set_xlabel('Hours Variance %')
ax.set_ylabel('Number of Jobs')
ax.set_title('Distribution of Hour Variances (clipped to -100% to +200%)')
ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
plt.tight_layout()
plt.show()

print(f"Jobs under budget (hours): {(quoted_jobs['Hours_Variance_Pct'] < 0).sum():,}")
print(f"Jobs over budget (hours):  {(quoted_jobs['Hours_Variance_Pct'] > 0).sum():,}")

## Client Analysis

In [ ]:
# Clients with worst margins (min 3 jobs)
client_agg = job_summary.groupby('Client').agg({
    'Job_No': 'count',
    'Billable_Amount': 'sum',
    'Actual_Cost': 'sum',
    'Profit': 'sum'
}).reset_index()
client_agg.columns = ['Client', 'Jobs', 'Revenue', 'Cost', 'Profit']
client_agg['Margin_Pct'] = (client_agg['Profit'] / client_agg['Revenue'] * 100).where(
    client_agg['Revenue'] > 0, 0
)

# Filter to clients with at least 3 jobs
client_agg = client_agg[client_agg['Jobs'] >= 3]
client_agg.nsmallest(15, 'Margin_Pct')

## FY-over-FY Comparison

In [ ]:
# Margin trends by fiscal year
fy_agg = job_summary.groupby('FY_Label').agg({
    'Job_No': 'count',
    'Quoted_Amount': 'sum',
    'Billable_Amount': 'sum',
    'Actual_Cost': 'sum',
    'Profit': 'sum'
}).reset_index()
fy_agg['Margin_Pct'] = (fy_agg['Profit'] / fy_agg['Billable_Amount'] * 100).where(
    fy_agg['Billable_Amount'] > 0, 0
)
fy_agg.columns = ['FY', 'Jobs', 'Quoted', 'Revenue', 'Cost', 'Profit', 'Margin %']
fy_agg.sort_values('FY')